## Load the Dataset

In this section, we import the dataset and prepare it for our models. The preprocessing pipeline includes splitting the data into **Training**, **Validation**, and **Test** sets.

Crucially, feature normalization will be applied by fitting the scaler **only on the training set**. These statistics are then applied to transform the validation and test sets. This approach is strictly necessary to avoid **data leakage**, ensuring that our model remains unbiased and effectively evaluates unseen data.

In [1]:
# using Pkg
# Ensure required packages (uncomment to install if needed)
# Pkg.add([
#     "MLJ", 
#     "MLJBase", 
#     "MLJModels", 
#     "MLJEnsembles", 
#     "MLJLinearModels", 
#     "DecisionTree", 
#     "MLJDecisionTreeInterface", 
#     "NaiveBayes", 
#     "EvoTrees", 
#     "CategoricalArrays", 
#     "Random",
#     "LIBSVM",           
#     "Plots",            
#     "MLJModelInterface", 
#     "CSV",              
#     "DataFrames",      
#     "MLJFlux", 
#     "UrlDownload",      
#     "XGBoost"    
# ])

include("Utils.jl")
using .Utils
using CSV
using DataFrames
using Statistics
using Random
using MLJ

Random.seed!(42)


TaskLocalRNG()

In [2]:
# 1. Load data from wdbc.data file
df = CSV.read("wdbc.data", DataFrame, header=false)

new_names = [
    "ID", "Diagnosis",
    # The Mean (first 10 features)
    "radius_mean", "texture_mean", "perimeter_mean", "area_mean", "smoothness_mean",
    "compactness_mean", "concavity_mean", "concave_points_mean", "symmetry_mean", "fractal_dimension_mean",
    # The Standard Error (next 10 features)
    "radius_se", "texture_se", "perimeter_se", "area_se", "smoothness_se",
    "compactness_se", "concavity_se", "concave_points_se", "symmetry_se", "fractal_dimension_se",
    # The "Worst" or Largest (last 10 features)
    "radius_worst", "texture_worst", "perimeter_worst", "area_worst", "smoothness_worst",
    "compactness_worst", "concavity_worst", "concave_points_worst", "symmetry_worst", "fractal_dimension_worst"
]
rename!(df, new_names)

# 4. Verify the changes
# Let's look at the first 3 rows and the new header
println("--- Updated DataFrame Headers ---")
first(df, 3)

# 2. Data separation (mimicking the Python script)
y = df[:, 2]
x = df[:, 3:end]

# 3. Metadata equivalent
println("--- Dataset Summary ---")
println("Total Rows: ", nrow(df))
println("Total Columns: ", ncol(df))
println("Target variable (y) shape: ", size(y))
println("Features matrix (X) shape: ", size(x))

println("\n--- Variable Information (First 5 Features) ---")
display(describe(x))
println("\n--- Target Distribution ---")
display(combine(groupby(DataFrame(Diagnosis=y), :Diagnosis), nrow))

--- Updated DataFrame Headers ---
--- Dataset Summary ---
Total Rows: 569
Total Columns: 32
Target variable (y) shape: (569,)
Features matrix (X) shape: (569, 30)

--- Variable Information (First 5 Features) ---

--- Target Distribution ---


Row,variable,mean,min,median,max,nmissing,eltype
,Symbol,Float64,Float64,Float64,Float64,Int64,DataType
1,radius_mean,14.1273,6.981,13.37,28.11,0,Float64
2,texture_mean,19.2896,9.71,18.84,39.28,0,Float64
3,perimeter_mean,91.969,43.79,86.24,188.5,0,Float64
4,area_mean,654.889,143.5,551.1,2501.0,0,Float64
5,smoothness_mean,0.0963603,0.05263,0.09587,0.1634,0,Float64
6,compactness_mean,0.104341,0.01938,0.09263,0.3454,0,Float64
7,concavity_mean,0.0887993,0.0,0.06154,0.4268,0,Float64
8,concave_points_mean,0.0489191,0.0,0.0335,0.2012,0,Float64
9,symmetry_mean,0.181162,0.106,0.1792,0.304,0,Float64


Row,Diagnosis,nrow
,String1,Int64
1,M,212
2,B,357


In [3]:
# This cell converts the input features 'x' into a Matrix format and identifies the unique classes
# from the target vector 'y'. It then performs One-Hot Encoding on the targets. Finally, it prints
# a summary of the raw data dimensions and basic statistics (mean, max) of the first feature
# column to verify the state of the data before any normalization is applied.

# 1. Convert to Matrix and Encode Targets (NOT NORMALIZED YET)
x = Matrix(x)

classes = unique(y)
y_encoded = Utils.oneHotEncoding(y, classes)

println("--- Data Summary (Raw Data) ---")
println("Features (X) dimension: ", size(x))
println("Targets (y_encoded) dimension: ", size(y_encoded))
# You will see real values (e.g., 14.12), not values between 0 and 1 yet
println("Mean of first feature column (Raw): ", mean(x[:, 1])) 
println("Max of first feature column (Raw): ", maximum(x[:, 1]))
println("Targets (first 5 encoded): \n", first(y_encoded, 5))

--- Data Summary (Raw Data) ---
Features (X) dimension: (569, 30)
Targets (y_encoded) dimension: (569, 1)
Mean of first feature column (Raw): 14.127291739894556
Max of first feature column (Raw): 28.11
Targets (first 5 encoded): 
Bool[1, 1, 1, 1, 1]


In [4]:
# This cell performs data splitting. It divides the data into training and test sets.
# Unlike the previous version, we do NOT create a global validation set here.
# IMPORTANT: In this step, we DO NOT normalize yet. We keep x_train and x_test 
# with their raw values. This allows the Cross-Validation process (in Utils) to 
# perform its own internal normalization per fold, ensuring strict rigorousness.

# 2. Splitting (Raw Data) - 80% Train, 20% Test
num_instances = size(x, 1)
test_ratio = 0.2 

# A. Get the indices (Using holdOut with 2 arguments returns Train and Test indices)
train_idx, test_idx = Utils.holdOut(num_instances, test_ratio)

# B. Create the subsets (RAW DATA - NOT NORMALIZED)
x_train = x[train_idx, :]
y_train = y_encoded[train_idx, :]

x_test = x[test_idx, :]
y_test = y_encoded[test_idx, :]

println("--- Split Summary (RAW DATA) ---")
println("Total instances: $num_instances")
println("Training set size: $(size(x_train, 1))")
println("Test set size: $(size(x_test, 1))")

# Verification: The values should be real numbers (e.g., > 1.0), not normalized yet
println("\n--- Raw Data Verification ---")
println("Max of first feature (Train): ", maximum(x_train[:, 1]))
println("Mean of first feature (Train): ", mean(x_train[:, 1]))

--- Split Summary (RAW DATA) ---
Total instances: 569
Training set size: 456
Test set size: 113

--- Raw Data Verification ---
Max of first feature (Train): 28.11
Mean of first feature (Train): 14.08660087719298


### Approach 1: All Dataset with CV

In this first approach, we will utilize **all available features** present in the dataset. 

This method serves as a baseline, allowing us to evaluate model performance using the complete set of variables without applying any feature selection or dimensionality reduction techniques.

In [5]:
# This cell sets up the configuration for "Approach 1" (using all data). It prepares the 
# target variables by converting them into vector format as required for model training. 
# It also initializes a dictionary to store the best hyperparameters found later. Finally, 
# it sets a fixed random seed and generates indices for 10-fold cross-validation to ensure 
# that the splits are reproducible.

# --- APPROACH 1 CONFIGURATION ---
println("--- INITIALIZING APPROACH: ALL DATA ---")

# 1. Prepare vector targets (required for MLJ)
# Assuming the positive class is 'true' in column 1
targets_train_vec = vec(y_train[:, 1]) 
targets_test_vec  = vec(y_test[:, 1])

# 2. Dictionary to save the best hyperparameters for each type
# (This is what we will use later for the Ensemble)
best_models = Dict()

# 3. Generate cross-validation indices (10 folds)
# Use a fixed seed so the folds are always the same
k_folds = 10
cv_indices = Utils.crossvalidation(targets_train_vec, k_folds)

println("All ready: Targets prepared and CV (10 folds) generated.")

--- INITIALIZING APPROACH: ALL DATA ---
All ready: Targets prepared and CV (10 folds) generated.


In [6]:
# This cell iterates through a list of different Neural Network topologies to find the best one.
# It uses a "Fast Mode" configuration with fewer epochs and a single execution per fold to
# significantly speed up the hyperparameter tuning process. The best performing topology
# (based on accuracy) is identified and saved to the best_models dictionary.

println("\n--- 1. ANN (Fast Mode) ---")
topologies = [[5], [10], [20], [5,5], [10,10], [20,10], [10,5], [30,15]]
best_ann_acc = 0.0
best_ann_params = nothing

for topo in topologies
    # Parameters optimized for speed
    hyperparameters = Dict(
        "topology" => topo,
        "maxEpochs" => 200,       # Few epochs to test quickly
        "learningRate" => 0.01,
        "validationRatio" => 0.2,
        "numExecutions" => 1      # <--- THIS MAKES IT TAKE SECONDS INSTEAD OF HOURS
    )
    
    metrics, _ = Utils.modelCompilation(:ANN, hyperparameters, (x_train, targets_train_vec), cv_indices)
    acc = metrics[1]
    
    # Print accuracy for the current topology
    println("Topology: $topo -> Acc: $(round(acc, digits=4))")
    
    if acc > best_ann_acc
        global best_ann_acc = acc
        global best_ann_params = hyperparameters
    end
end
println(">> BEST ANN: $(best_ann_params["topology"]) | Acc: $(round(best_ann_acc, digits=4))")
best_models[:ANN] = best_ann_params


--- 1. ANN (Fast Mode) ---


┌ Warning: Layer with Float32 parameters got Float64 input.
│   The input will be converted, but any earlier layers may be very slow.
│   layer = Dense(30 => 5, σ)
│   summary(x) = 30×328 adjoint(::Matrix{Float64}) with eltype Float64
└ @ Flux C:\Users\Hugo\.julia\packages\Flux\uRn8o\src\layers\stateless.jl:60


Topology: [5] -> Acc: 0.8953
Topology: [10] -> Acc: 0.9648
Topology: [20] -> Acc: 0.9671
Topology: [5, 5] -> Acc: 0.8886
Topology: [10, 10] -> Acc: 0.9737
Topology: [20, 10] -> Acc: 0.9782
Topology: [10, 5] -> Acc: 0.9782
Topology: [30, 15] -> Acc: 0.9739
>> BEST ANN: [20, 10] | Acc: 0.9782


Dict{String, Any} with 5 entries:
  "maxEpochs"       => 200
  "learningRate"    => 0.01
  "topology"        => [20, 10]
  "validationRatio" => 0.2
  "numExecutions"   => 1

In [7]:
# This cell trains Support Vector Machine (SVM) models using 8 different configurations. 
# It iterates through various combinations of kernel types (RBF, Linear, Poly, Sigmoid) 
# and regularization parameters 'C'. For each configuration, it evaluates performance using 
# cross-validation, identifies the best configuration based on accuracy, and stores the 
# winning parameters in the best_models dictionary.

println("\n--- 2. Training SVM ---")

# 8 Configurations
configs_svm = [
    ("rbf", 1.0), ("rbf", 10.0), ("rbf", 100.0),
    ("linear", 1.0), ("linear", 10.0),
    ("poly", 1.0), ("poly", 10.0),
    ("sigmoid", 1.0)
]

best_svm_acc = 0.0
best_svm_params = nothing

for (k, c) in configs_svm
    params = Dict("kernel" => k, "C" => c)
    metrics, _ = Utils.modelCompilation(:SVC, params, (x_train, targets_train_vec), cv_indices)
    acc = metrics[1]
    
    # Print accuracy for the current configuration
    println("Kernel: $k, C: $c -> Acc: $(round(acc, digits=4))")
    
    if acc > best_svm_acc
        global best_svm_acc = acc
        global best_svm_params = params
    end
end

println(">> SVM WINNER: Kernel $(best_svm_params["kernel"]), C=$(best_svm_params["C"]) with Acc: $best_svm_acc")
best_models[:SVM] = best_svm_params


--- 2. Training SVM ---
Kernel: rbf, C: 1.0 -> Acc: 0.9647
Kernel: rbf, C: 10.0 -> Acc: 0.9759
Kernel: rbf, C: 100.0 -> Acc: 0.9671
Kernel: linear, C: 1.0 -> Acc: 0.9758
Kernel: linear, C: 10.0 -> Acc: 0.9694
Kernel: poly, C: 1.0 -> Acc: 0.956
Kernel: poly, C: 10.0 -> Acc: 0.9691
Kernel: sigmoid, C: 1.0 -> Acc: 0.9626
>> SVM WINNER: Kernel rbf, C=10.0 with Acc: 0.9758893280632412


Dict{String, Any} with 2 entries:
  "C"      => 10.0
  "kernel" => "rbf"

In [8]:
# This cell trains Decision Tree classifiers to find the optimal tree depth.
# It iterates through a predefined list of maximum depths (from 3 to 12) and evaluates
# each using the model compilation utility with cross-validation. It prints the accuracy
# for each depth and stores the best-performing configuration in the best_models dictionary.

println("\n--- 3. Training Decision Trees ---")

# 6 Depths
depths = [3, 4, 5, 7, 9, 12]

best_dt_acc = 0.0
best_dt_params = nothing

for d in depths
    params = Dict("max_depth" => d)
    metrics, _ = Utils.modelCompilation(:DecisionTreeClassifier, params, (x_train, targets_train_vec), cv_indices)
    acc = metrics[1]
    
    println("Depth: $d -> Acc: $(round(acc, digits=4))")
    
    if acc > best_dt_acc
        global best_dt_acc = acc
        global best_dt_params = params
    end
end

println(">> DT WINNER: Depth $(best_dt_params["max_depth"]) with Acc: $best_dt_acc")
best_models[:DecisionTree] = best_dt_params


--- 3. Training Decision Trees ---
Depth: 3 -> Acc: 0.9539
Depth: 4 -> Acc: 0.9473
Depth: 5 -> Acc: 0.9518
Depth: 7 -> Acc: 0.9518
Depth: 9 -> Acc: 0.9518
Depth: 12 -> Acc: 0.9518
>> DT WINNER: Depth 3 with Acc: 0.9538581466842336


Dict{String, Int64} with 1 entry:
  "max_depth" => 3

In [9]:
# This cell trains k-Nearest Neighbors (kNN) classifiers by testing 6 different values for 'k'
# (number of neighbors). It evaluates each configuration using cross-validation, tracks the 
# best performing one based on accuracy, and stores the winning parameter in the best_models 
# dictionary. It explicitly adjusts the dictionary key format to ensure compatibility with 
# the Ensemble module used later.

println("\n--- 4. Training kNN ---")

# 6 k values
k_values = [1, 3, 5, 7, 9, 11]

best_knn_acc = 0.0
best_knn_params = nothing

for k in k_values
    params = Dict("n_neighbors" => k)
    metrics, _ = Utils.modelCompilation(:KNeighborsClassifier, params, (x_train, targets_train_vec), cv_indices)
    acc = metrics[1]
    
    println("k: $k -> Acc: $(round(acc, digits=4))")
    
    if acc > best_knn_acc
        global best_knn_acc = acc
        global best_knn_params = params
    end
end

# Key adjustment for compatibility with the Ensemble module (requires :K, not :n_neighbors)
best_models[:kNN] = Dict("K" => best_knn_params["n_neighbors"])

println(">> kNN WINNER: k=$(best_models[:kNN]["K"]) with Acc: $best_knn_acc")


--- 4. Training kNN ---
k: 1 -> Acc: 0.9582
k: 3 -> Acc: 0.9603
k: 5 -> Acc: 0.9602
k: 7 -> Acc: 0.9624
k: 9 -> Acc: 0.9601
k: 11 -> Acc: 0.9668
>> kNN WINNER: k=11 with Acc: 0.9668006148440931


In [10]:
# This cell trains an Ensemble model combining the four previously optimized algorithms: SVM, 
# Decision Tree, kNN, and ANN. It defines a helper function to convert parameter keys from 
# strings to symbols to match the requirements of the training function. The ensemble is 
# configured to use 'hard voting' (majority rule) and is evaluated using the cross-validation 
# indices generated earlier. Finally, it reports the average accuracy of the ensemble.

println("\n--- 5. Training Ensemble (4 Models: SVM, DT, kNN, ANN) ---")

# Helper function to convert String keys to Symbol
# (Necessary because Utils.trainClassEnsemble looks for :topology, not "topology")
function string_to_symbol_keys(d::Dict)
    return Dict(Symbol(k) => v for (k, v) in d)
end

# 1. Define the 4 estimators
estimators_list = [:SVM, :DecisionTree, :kNN, :ANN]

# 2. Prepare parameters (converting keys to Symbols)
params_list = [
    string_to_symbol_keys(best_models[:SVM]),
    string_to_symbol_keys(best_models[:DecisionTree]),
    string_to_symbol_keys(best_models[:kNN]),
    string_to_symbol_keys(best_models[:ANN])
]

println("Training ensemble with the 4 models... (This might take a bit longer due to the ANN)")

# 3. Train the ensemble
ensemble_res = Utils.trainClassEnsemble(
    estimators_list, 
    params_list, 
    Dict(:voting => :hard), # Majority voting
    (x_train, y_train),     # Use original targets (one-hot/matrix)
    cv_indices
)

ensemble_acc = ensemble_res.mean_metric
println(">> ENSEMBLE RESULT (CV 4 Models): Mean Acc = $(round(ensemble_acc, digits=4))")


--- 5. Training Ensemble (4 Models: SVM, DT, kNN, ANN) ---
Training ensemble with the 4 models... (This might take a bit longer due to the ANN)
>> ENSEMBLE RESULT (CV 4 Models): Mean Acc = 0.9672


In [11]:
# This cell performs the final evaluation of the best model found. It compares the 
# cross-validation accuracies to select the "winner". 
# CRUCIAL STEP: Before training the winner, it calculates normalization parameters 
# using the FULL training set (x_train) and applies them to create normalized 
# versions of the training and test sets. Then, it retrains the winner.

println("\n" * "="^50)
println(" FINAL APPROACH EVALUATION (TEST SET)")
println("="^50)

# --- NECESSARY IMPORTS ---
import LIBSVM
import DecisionTree
import NearestNeighborModels
import MLJFlux
using Flux 
using CategoricalArrays 
using Random
using MLJ

# 1. Compare cross-validation results to choose the champion
accuracies = [best_ann_acc, best_svm_acc, best_dt_acc, best_knn_acc, ensemble_acc]
model_names = ["ANN", "SVM", "DecisionTree", "kNN", "Ensemble"]

best_idx = argmax(accuracies)
winner_name = model_names[best_idx]
println("The winning model in Cross-Validation was: $winner_name (Acc: $(round(accuracies[best_idx], digits=4)))")

# ---------------------------------------------------------
# 2. DATA NORMALIZATION FOR FINAL TRAINING
# ---------------------------------------------------------
println("\n--- Normalizing data for final training ---")

# Create copies to hold the normalized data
x_train_final = copy(x_train) # Starts as raw
x_test_final  = copy(x_test)  # Starts as raw

# Calculate parameters based ONLY on Training Data
normParams = Utils.calculateMinMaxNormalizationParameters(x_train_final)

# Apply normalization
Utils.normalizeMinMax!(x_train_final, normParams)
Utils.normalizeMinMax!(x_test_final, normParams)

println("Data normalized. Mean feature 1 (Train): $(mean(x_train_final[:,1]))")

println("\n--- Training the Winner with ALL Train Set and Evaluating on Test Set ---")

# Set seed for reproducibility of the final training
Random.seed!(42)

y_pred_final = nothing

if winner_name == "SVM"
    p = best_models[:SVM]
    kernel_map = Dict(
        "linear" => LIBSVM.Kernel.Linear, "rbf" => LIBSVM.Kernel.RadialBasis,
        "poly" => LIBSVM.Kernel.Polynomial, "sigmoid" => LIBSVM.Kernel.Sigmoid
    )
    chosen_kernel = kernel_map[p["kernel"]]
    model_type = MLJ.@load ProbabilisticSVC pkg=LIBSVM verbosity=0
    model = model_type(kernel=chosen_kernel, cost=Float64(p["C"]))
    # Use x_train_final
    mach = machine(model, MLJ.table(x_train_final), categorical(targets_train_vec))
    fit!(mach, verbosity=0)
    y_pred_final = mode.(predict(mach, MLJ.table(x_test_final)))

elseif winner_name == "DecisionTree"
    p = best_models[:DecisionTree]
    model_type = MLJ.@load DecisionTreeClassifier pkg=DecisionTree verbosity=0
    model = model_type(max_depth=p["max_depth"], rng=Random.MersenneTwister(42))
    # Use x_train_final
    mach = machine(model, MLJ.table(x_train_final), categorical(targets_train_vec))
    fit!(mach, verbosity=0)
    y_pred_final = mode.(predict(mach, MLJ.table(x_test_final)))

elseif winner_name == "kNN"
    p = best_models[:kNN]
    model_type = MLJ.@load KNNClassifier pkg=NearestNeighborModels verbosity=0
    k_val = haskey(p, "K") ? p["K"] : p["n_neighbors"]
    model = model_type(K=k_val)
    # Use x_train_final
    mach = machine(model, MLJ.table(x_train_final), categorical(targets_train_vec))
    fit!(mach, verbosity=0)
    y_pred_final = mode.(predict(mach, MLJ.table(x_test_final)))

elseif winner_name == "ANN"
    println("Retraining ANN with Utils...")
    println("(Creating an internal 20% validation split from x_train_final for Early Stopping)")
    p = best_models[:ANN]
    
    # --- INTERNAL SPLIT FOR ANN ---
    # We split x_train_final into 80% train / 20% validation just for this training
    n_train_final = size(x_train_final, 1)
    t_idx, v_idx = Utils.holdOut(n_train_final, 0.2) # 20% for validation
    
    x_t_ann = x_train_final[t_idx, :]
    y_t_ann = y_train[t_idx, :]
    
    x_v_ann = x_train_final[v_idx, :]
    y_v_ann = y_train[v_idx, :]
    
    # Train using the sub-split
    ann, _, _, _ = Utils.trainClassANN(
        p["topology"],
        (x_t_ann, y_t_ann), 
        validationDataset=(x_v_ann, y_v_ann), # Internal validation set passed here
        maxEpochs=p["maxEpochs"],
        learningRate=p["learningRate"]
    )
    
    # Manual prediction on x_test_final
    y_pred_raw = ann(x_test_final')'
    y_pred_bool_manual = vec(Utils.classifyOutputs(y_pred_raw))
    Utils.printConfusionMatrix(y_pred_bool_manual, targets_test_vec)
    y_pred_final = nothing 

elseif winner_name == "Ensemble"
    println("Rebuilding Ensemble...")
    # 1. Base SVM
    p_svm = best_models[:SVM]
    svm_k = Dict("linear"=>LIBSVM.Kernel.Linear, "rbf"=>LIBSVM.Kernel.RadialBasis, "poly"=>LIBSVM.Kernel.Polynomial, "sigmoid"=>LIBSVM.Kernel.Sigmoid)[p_svm["kernel"]]
    SVC = MLJ.@load ProbabilisticSVC pkg=LIBSVM verbosity=0
    m1 = SVC(kernel=svm_k, cost=Float64(p_svm["C"]))
    # 2. Base DT
    p_dt = best_models[:DecisionTree]
    DTC = MLJ.@load DecisionTreeClassifier pkg=DecisionTree verbosity=0
    m2 = DTC(max_depth=p_dt["max_depth"])
    # 3. Base kNN
    p_knn = best_models[:kNN]
    KNN = MLJ.@load KNNClassifier pkg=NearestNeighborModels verbosity=0
    m3 = KNN(K=p_knn["K"])
    # 4. Base ANN
    p_ann = best_models[:ANN]
    NNC = MLJ.@load NeuralNetworkClassifier pkg=MLJFlux verbosity=0
    builder = MLJFlux.Short(n_hidden=p_ann["topology"][1], σ=Flux.relu) 
    m4 = NNC(builder=builder, epochs=200) 
    
    ensemble_model = Utils.VotingClassifier(models=[m1, m2, m3, m4], voting=:hard)
    # Use x_train_final
    mach = machine(ensemble_model, MLJ.table(x_train_final), categorical(targets_train_vec))
    fit!(mach, verbosity=0)
    y_pred_final = predict_mode(mach, MLJ.table(x_test_final))
end

if !isnothing(y_pred_final)
    println("\n--- FINAL CONFUSION MATRIX (TEST SET) ---")
    y_pred_bool = [x == "true" || x == true for x in MLJ.unwrap.(y_pred_final)]
    Utils.printConfusionMatrix(y_pred_bool, targets_test_vec)
end


 FINAL APPROACH EVALUATION (TEST SET)
The winning model in Cross-Validation was: ANN (Acc: 0.9782)

--- Normalizing data for final training ---
Data normalized. Mean feature 1 (Train): 0.33629612746429

--- Training the Winner with ALL Train Set and Evaluating on Test Set ---
Retraining ANN with Utils...
(Creating an internal 20% validation split from x_train_final for Early Stopping)
Confusion Matrix:
[70 0; 2 41]

Accuracy: 0.9823008849557522
Error rate: 0.017699115044247787
Sensitivity: 0.9534883720930233
Specificity: 1.0
PPV: 1.0
NPV: 0.9722222222222222
F1-score: 0.9761904761904763
